In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (6800 Fisher Island Dr #6803, Miami Beach, FL ...
2     (7939 East Dr #0, North Bay Village, FL 33141,...
3     (350 Ocean Dr #802n, Key Biscayne, FL 33149, U...
4     (5231 Fisher Island Dr #5231, Miami Beach, FL ...
5     (15701 Collins Ave #2005, North Miami Beach, F...
6     (FAENA HOUSE, 3315 Collins Ave #8c, Miami Beac...
7     (1 Collins Ave Unit 408, Miami Beach, FL 33139...
8     (430 Grand Bay Dr APT 603, Key Biscayne, FL 33...
9     (Santa Maria, 1643 Brickell Ave APT 3404, Miam...
10    (16901 Collins Ave #3102, Sunny Isles Beach, F...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Buyer Agent,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Fisher Island\r6800 Fisher Island PH 6803\rMia...,3/29/2021,3/31/2022,248,"$30,000,000.00",6560,"$4,573.17",Dora Puig,Anthony Lauria,Fisher Island 6800 Fisher Island PH 6803 Miami...,Fisher Island,6800 Fisher Island PH 6803 Miami Beach,"(6800 Fisher Island Dr #6803, Miami Beach, FL ...","(25.7623668, -80.14735929999999, 0.0)",25.762367,-80.147359,0.0
2,Adagio\r7939 East Dr 0\rNorth Bay Village,5/17/2018,3/29/2022,1398,"$17,000,000.00",33596,$506.01,Susan Gale,Susan Gale,Adagio 7939 East Dr 0 North Bay Village,Adagio,7939 East Dr 0 North Bay Village,"(7939 East Dr #0, North Bay Village, FL 33141,...","(25.8533558, -80.15714050000001, 0.0)",25.853356,-80.157141,0.0
3,Oceana Key Biscayne\r350 Ocean Dr 802N\rKey Bi...,8/25/2021,3/29/2022,152,"$8,800,000.00",3600,"$2,444.44",Giulietta Ulloa,Barbara Lamar PA,Oceana Key Biscayne 350 Ocean Dr 802N Key Bisc...,Oceana Key Biscayne,350 Ocean Dr 802N Key Biscayne,"(350 Ocean Dr #802n, Key Biscayne, FL 33149, U...","(25.694538, -80.15759829999999, 0.0)",25.694538,-80.157598,0.0
4,Fisher Island\r5231 Fisher Island Dr 5231\rMia...,11/5/2021,3/31/2022,45,"$8,400,000.00",4010,"$2,094.76",Lazaro Rivero,Lazaro Rivero,Fisher Island 5231 Fisher Island Dr 5231 Miami...,Fisher Island,5231 Fisher Island Dr 5231 Miami Beach,"(5231 Fisher Island Dr #5231, Miami Beach, FL ...","(25.7626891, -80.1484152, 0.0)",25.762689,-80.148415,0.0
5,Ritz Carlton Sunny Isles\r15701 Collins Ave 20...,1/12/2022,3/28/2022,36,"$6,550,000.00",3204,"$2,044.32",Gabriela Gonzalez Carrillo,Susan Rindley,Ritz Carlton Sunny Isles 15701 Collins Ave 200...,Ritz Carlton Sunny Isles,15701 Collins Ave 2005 Sunny Isles Beach,"(15701 Collins Ave #2005, North Miami Beach, F...","(25.9220146, -80.1217034, 0.0)",25.922015,-80.121703,0.0
6,Faena House\r3315 Collins Ave 8C\rMiami Beach,8/31/2021,4/1/2022,179,"$5,800,000.00",3428,"$2,591.60",Oliver Lloyd,Rachael Taylor,Faena House 3315 Collins Ave 8C Miami Beach,Faena House,3315 Collins Ave 8C Miami Beach,"(FAENA HOUSE, 3315 Collins Ave #8c, Miami Beac...","(25.8078488, -80.1232302, 0.0)",25.807849,-80.123230,0.0
7,One Ocean Condo\r1 Collins Ave 408\rMiami Beach,12/16/2021,3/29/2022,53,"$5,600,000.00",3094,"$1,809.95",Toni Schrager,Marci Declaris,One Ocean Condo 1 Collins Ave 408 Miami Beach,One Ocean Condo,1 Collins Ave 408 Miami Beach,"(1 Collins Ave Unit 408, Miami Beach, FL 33139...","(25.768079, -80.1339531, 0.0)",25.768079,-80.133953,0.0
8,Grand Bay Residences\r430 Grand Bay Dr 603\rKe...,10/21/2021,3/30/2022,103,"$4,100,000.00",3630,"$1,129.48",Natalia Salgado,Maria Guinea,Grand Bay Residences 430 Grand Bay Dr 603 Key ...,Grand Bay Residences,430 Grand Bay Dr 603 Key Biscayne,"(430 Grand Bay Dr APT 603, Key Biscayne, FL 33...","(25.6930959, -80.1579994, 0.0)",25.693096,-80.157999,0.0
9,Santa Maria\r1643 Brickell Ave 3404\rMiami,11/23/2021,3/28/2022,83,"$3,850,000.00",4030,$955.34,Luis Gonell,Erika Cedeno,Santa Maria 1643 Brickell Ave 3404 Miami,Santa Maria,1643 Brickell Ave 3404 Miami,"(Santa Maria, 1643 Brickell Ave APT 3404, Miam...","(25.755259, -80.1947467, 0.0)",25.755259,-80.194747,0.0
10,Jade Signature\r16901 Collins Ave 3102\rSunny ...,7/5/2021,3/31/2022,182,"$3,618,026.00",2494,"$1,450.69",Enrique Leizorek,Eyal Golan PA,Jade Signature 16901 Collins Ave 3102 Sunny Is...,Jade Signature,16901 Collins Ave 3102 Sunny Isles Beach,"(16901 Collins Ave #3102, Sunny Isles Beach, F...","(25.9327557, -80.1210054, 0.0)",25.932756,-80.121005,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
#df.at[1,'Buyer Agent']=('Dora Puig, Carl Gambino')

In [11]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Buyer Agent',
       'geo_address', 'building_name', 'address_only', 'loc', 'point', 'lat',
       'lon', 'altitude'],
      dtype='object')

In [14]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('March 27th - April 2nd')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [15]:
m.save('index.html')